In [ ]:
%whos

In [ ]:
from os import listdir
from os.path import isfile, join
mypath = '/opt/notebooks/AIOps/parquet'
onlyfiles = [f for f in listdir(mypath) if f.count('basic_data') == 0 and isfile(join(mypath, f))]
print(onlyfiles)

In [ ]:
fileName = join(mypath, onlyfiles[0])
file = pd.read_parquet(fileName)
print(fileName)
print(file.info())

In [ ]:
file.head()

In [ ]:
#read selected columns from files
import pandas as pd
data = pd.DataFrame()
for f in onlyfiles:
    file = join(mypath, f)
    day = pd.read_parquet(file, columns=['timestamp', 'span_destination_service_name', 'transaction_id', 'span_id', 'span_action', 'span_type', 'span_subtype', 'span_name', 'span_duration_us'])
    data = data.append(day)
    print(file)
print('Done')

In [ ]:
data.info()

In [ ]:
data.to_parquet('/opt/notebooks/AIOps/parquet/basic_data.parquet')

In [ ]:
import pandas as pd
#data = pd.read_parquet('/opt/notebooks/AIOps/parquet/basic_data.parquet')

In [ ]:
pd.to_datetime(data.loc[:, 'timestamp']).dt.date.unique().size

In [ ]:
#60% od the data is logging
data_no_logging = data[(data['span_action'] != 'send') & (data['span_type'] != 'messaging') & (data['span_subtype'] != 'kafka')]
data_no_logging.head(20)

In [ ]:
print(data.shape, data_no_logging.shape)

In [ ]:
pd.to_datetime(data_no_logging.loc[:, 'timestamp']).dt.date.unique().size

In [ ]:
data_no_logging.to_parquet('/opt/notebooks/AIOps/parquet/basic_data_no_logging.parquet')

In [ ]:
import pandas as pd
data_no_logging = pd.read_parquet('/opt/notebooks/AIOps/parquet/basic_data_no_logging.parquet')

In [ ]:
data_no_logging.set_index('span_destination_service_name', inplace=True, drop=False)

In [ ]:
#get unique service names
service_names = data_no_logging.loc[:, 'span_destination_service_name'].unique()
display(service_names)

In [ ]:
#unknown service
#data_no_logging[data_no_logging['span_destination_service_name'] ==  service_names[9]].head(20)

In [ ]:
grouped_by_service = data_no_logging.groupby(level = 'span_destination_service_name')
count_by_service = grouped_by_service.size()
display(count_by_service)

In [ ]:
#get unique span types
data_no_logging.loc[:, 'span_type'].unique()

In [ ]:
#get unique span subtypes
data_no_logging.loc[:, 'span_subtype'].unique()

In [ ]:
#get unique span name
data_no_logging.loc[:, 'span_name'].unique()

In [ ]:
data_no_logging.loc[data_no_logging['span_destination_service_name'] == '', 'span_name'].shape

In [ ]:
data_no_logging.loc[data_no_logging['span_destination_service_name'] == '', 'span_name'].unique()

In [ ]:
#update service name for rows that have service name empty
data_no_logging.loc[(data_no_logging['span_destination_service_name'] == '') & (data_no_logging['span_name'].str.contains('productInventory') == 1), 'span_destination_service_name'] = 'productInventory'
data_no_logging.loc[(data_no_logging['span_destination_service_name'] == '') & (data_no_logging['span_name'].str.contains('profileManagement') == 1), 'span_destination_service_name'] = 'profileManagement'
data_no_logging.loc[(data_no_logging['span_destination_service_name'] == '') & (data_no_logging['span_name'].str.contains('am-publicweb') == 1), 'span_destination_service_name'] = 'am-publicweb'
data_no_logging.loc[(data_no_logging['span_destination_service_name'] == '') & (data_no_logging['span_name'].str.contains('api-gw') == 1), 'span_destination_service_name'] = 'api-gw'
data_no_logging.loc[(data_no_logging['span_destination_service_name'] == '') & (data_no_logging['span_name'].str.contains('services.mw') == 1), 'span_destination_service_name'] = 'services.mw'

In [ ]:
#analysis for service 1
service1 = service_names[1]
service1_data = data_no_logging[data_no_logging['span_destination_service_name'] == service1]
display(service1_data.shape)
service1_data.to_parquet('/opt/notebooks/AIOps/parquet/service1_basic_data.parquet')

In [ ]:
service1_data.loc[:, 'date'] = pd.to_datetime(service1_data.loc[:, 'timestamp']).dt.date
service1_data.loc[:, 'day'] = pd.to_datetime(service1_data.loc[:, 'timestamp']).dt.day_of_week
service1_data.loc[:, 'hour'] = pd.to_datetime(service1_data.loc[:, 'timestamp']).dt.hour
service1_data.loc[:, 'minute'] = pd.to_datetime(service1_data.loc[:, 'timestamp']).dt.minute
service1_data.loc[:, 'second'] = pd.to_datetime(service1_data.loc[:, 'timestamp']).dt.second

In [ ]:
service1_data.head()

In [ ]:
service1_data['date'].unique().size